In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
import nltk as nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string
from nltk.tokenize import word_tokenize

In [2]:
df=pd.read_excel('data_sizing_bin1.xlsx')
df=df.drop(['Unnamed: 0',
 'Unnamed: 0.1',
 'TaskID',
'BudgetLineID',
 'ProjectID',
 'RegionID','CurrentProcess',
 'PCPNumber',
 'CR',
 'PCPType','TaskStatus',
 'DescopeReason','OOM',
 'OOMStr', 'CategoryKeyword',
 'LateUATJustification',
 'OOMForumReview',
 'OOMForumApproved',
 'ExpiryFlag',
 'DateOpen',
 'OpenYear',
 'AssessmentMode',
 'OOMForumComments',
 'OOMComments',
 'AssessmentInfoComments',
 'PCPComments','PCPCategory',
 'Itemization',
 'UpdatedDate','Sizing_cpy','UATDuration','AssessmentCategory','DomainID','AreaID','FundingSolution','totalwords_RequirementDescription','Sizing'],axis=1)
cname=list(df.columns)
df=df.reset_index(drop = True)
df['Sizing_bins']=df['Sizing_bins'].replace(['Very Small','Small','Medium','Large'], [0,1,2,3])
#df=df.drop(['Sizing_bins'],axis=1)
print('\n')
print('Null Value list \n',df.isnull().sum())
print('\n') 
print(df.info())



Null Value list 
 Title                        0
Domain                       0
Area                      2444
BudgetLine                1728
ProjectName                  0
ProjectDescription           0
Region                    1441
RequirementDescription      71
SolutionSummary           3994
SolutionDescription       3790
SolutionAssumption        6955
ProductName               3093
RecommendedPricing         264
Sizing_bins               3765
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 14 columns):
Title                     11113 non-null object
Domain                    11113 non-null object
Area                      8669 non-null object
BudgetLine                9385 non-null object
ProjectName               11113 non-null object
ProjectDescription        11113 non-null object
Region                    9672 non-null object
RequirementDescription    11042 non-null object
SolutionSummary           7119 non-null o

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 14 columns):
Title                     11113 non-null object
Domain                    11113 non-null object
Area                      8669 non-null object
BudgetLine                9385 non-null object
ProjectName               11113 non-null object
ProjectDescription        11113 non-null object
Region                    9672 non-null object
RequirementDescription    11042 non-null object
SolutionSummary           7119 non-null object
SolutionDescription       7323 non-null object
SolutionAssumption        4158 non-null object
ProductName               8020 non-null object
RecommendedPricing        10849 non-null object
Sizing_bins               7348 non-null float64
dtypes: float64(1), object(13)
memory usage: 1.2+ MB


In [4]:
df=df.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 14 columns):
Title                     11113 non-null object
Domain                    11113 non-null object
Area                      11113 non-null object
BudgetLine                11113 non-null object
ProjectName               11113 non-null object
ProjectDescription        11113 non-null object
Region                    11113 non-null object
RequirementDescription    11113 non-null object
SolutionSummary           11113 non-null object
SolutionDescription       11113 non-null object
SolutionAssumption        11113 non-null object
ProductName               11113 non-null object
RecommendedPricing        11113 non-null object
Sizing_bins               11113 non-null object
dtypes: object(14)
memory usage: 1.2+ MB


In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUMBERS =re.compile("[\d+]")
STOPWORDS = set(stopwords.words('english'))


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REMOVE_NUMBERS.sub('',text) # replace REMOVE_NUMBERS by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)# delete stopwors from text
    #text = ' '.join(word for word in text.split() if word not in common_words)
    return text

In [6]:
cname=list(df.columns)
for i in cname:
    df[i]=df[i].apply(clean_text)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:333: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/agile/browse/HITPP-1238" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/agile/browse/HITPP-1239" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  Mar

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/PDA/ST+89616+BR%3A+TKT%3A+Revalidation+control+for+name+ID+code+and+associated+condition" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/PDA/ST+89671+CI%3A+TKT%3A+Enhancement+of+Revalidation+Function" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResembl

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/MERTPC/PCP%2315491994+-+Hotel+chain+provider+in+the+Teaser+Criteria" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/RDAIRTDFBTF/Feasibility+Study+-+CP+16075543" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.am

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/RDAIRTDFBTF/13484525%3A+Feasibility+study" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/ARDWSOL/SWA%3A0754%3AARD%3AAdd+SGA+Banner+in+ARD+shopping+and+availability+screens" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/display/HITREPORTING/106045+%5BCLARIFIED%5D+-+Migration+Status+Report" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluence/pages/viewpage.action?pageId=913927071" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://rndwww.nce.amadeus.net/confluenc

In [7]:
corpus_df = df['Title']+ " " + df['Domain'] + " " + df['Area'] + " " + df['BudgetLine'] + " " + df['ProjectName'] + " " + df['ProjectDescription'] +" "+ df['Region'] + " " +df['RequirementDescription'] + df['SolutionSummary'] +' '+df['SolutionDescription']+' '+df['SolutionAssumption']+' '+df['ProductName']+' '+df['RecommendedPricing']

In [8]:
corpus_df = corpus_df.str.split(' ')

In [9]:
cleaned = []

for text in corpus_df:
    text = [x.strip(string.punctuation) for x in text]
    cleaned.append(text)

cleaned[0]

text_spam = [" ".join(text) for text in cleaned]
final_text = " ".join(text_spam)
final_text[:500]

'time spent reporting airline bespoke cp detailed assessment srb gui airline private development  amadeus nan time spent reporting airline bespoke cp detailed assessmenttime spent reporting airline bespoke cp detailed assessment task automatically created importing cr winaproach time spent reporting airline bespoke cp detailed assessment altea time rptg assessment time spent reporting airline bespoke cp oom assessment srb oui airline private development  amadeus nan time spent reporting airline b'

In [10]:
tok_final_text=word_tokenize(final_text)

In [11]:
from collections import Counter
word_counts = Counter(tok_final_text)
words = list(word_counts.most_common(5000))
words.pop(0)

('nan', 19725)

In [12]:
words

[('sold', 18811),
 ('added', 15656),
 ('new', 11728),
 ('airlines', 11027),
 ('cp', 11021),
 ('fee', 10472),
 ('fees', 10027),
 ('customer', 9845),
 ('requesting', 9799),
 ('airline', 9711),
 ('change', 7881),
 ('dcs', 7273),
 ('development', 6990),
 ('flight', 6875),
 ('amadeus', 6277),
 ('solution', 6261),
 ('created', 5987),
 ('code', 5974),
 ('task', 5519),
 ('automatically', 5214),
 ('pnr', 5153),
 ('document', 4886),
 ('apac', 4764),
 ('altea', 4607),
 ('importing', 4526),
 ('data', 4393),
 ('number', 4257),
 ('n', 4220),
 ('time', 4197),
 ('cm', 4152),
 ('information', 4140),
 ('would', 4061),
 ('res', 3988),
 ('update', 3752),
 ('display', 3701),
 ('one', 3673),
 ('addedusage', 3641),
 ('airlinesmaintenance', 3596),
 ('date', 3582),
 ('fare', 3569),
 ('seat', 3475),
 ('use', 3469),
 ('process', 3450),
 ('air', 3433),
 ('need', 3424),
 ('rule', 3410),
 ('boarding', 3392),
 ('customerhow', 3380),
 ('check', 3357),
 ('add', 3346),
 ('private', 3220),
 ('scope', 3194),
 ('requireme

In [13]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
print(len(tokenizer))  # 28996
tokenizer.add_tokens(words)
print(len(tokenizer)) 
model.resize_token_embeddings(len(tokenizer)) 

30522
35521


Embedding(35521, 768)

In [14]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from tqdm import tqdm
#from early_stopping import EarlyStopping
import string

In [15]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        data = data.reset_index()
        self.tokenizer = tokenizer
        self.text = data.text
        self.targets = data.target
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(inputs["token_type_ids"], dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [16]:
class BERTForRegression(torch.nn.Module):
    def __init__(self, layer_size=768, dropout=0.3):
        super(BERTForRegression, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.hidden_layer = torch.nn.Linear(768, layer_size)
        self.dropout = torch.nn.Dropout(dropout)
        self.classifier = torch.nn.Linear(layer_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # BERT "pooling": https://github.com/huggingface/transformers/issues/1535
        bert_output = output_1[0][:, 0]
        x = self.hidden_layer(bert_output)
        x = torch.nn.ReLU()(x)
        # With dropout
        x = self.dropout(x)
        # Final layer
        x = self.classifier(x)
        return torch.nn.ReLU()(x)

    def eval_with_dict(self, data):
        return self(
            data['input_ids'].to(device, dtype=torch.long),
            data['attention_mask'].to(device, dtype=torch.long),
            data['token_type_ids'].to(device, dtype=torch.long)
        ).squeeze(1)

    def eval_with_loader(self, loader):
        self.eval()
        fin_targets = []
        fin_outputs = []
        with torch.no_grad():
            for _, data in enumerate(loader, 0):
                predictions = self.eval_with_dict(data)
                fin_targets.extend(data['targets'].tolist())
                fin_outputs.extend(torch.sigmoid(predictions).to("cpu").detach().numpy().tolist())
        return fin_outputs, fin_targets


def compute_abs_error(predictions, targets, scaler):
    return torch.abs(
        untransform(predictions, scaler) - untransform(targets, scaler)
    ).sum().data


def untransform(y, scaler):
    return torch.exp(
        y * (scaler.data_max_[0] - scaler.data_min_[0]) + scaler.data_min_[0]
    ) - 1

In [17]:
class Trainer:
    def __init__(self, dataset, tokenizer, learning_rate, patience, train_batch_size, test_batch_size):
        self.tokenizer = tokenizer
        self.learning_rate = learning_rate
        self.patience = patience
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size
        self.max_length = tokenizer.model_max_length
        self.loss = torch.nn.MSELoss()

        self.prepare_data(dataset)

    def prepare_data(self, dataset):
        train_dataset, test_dataset = train_test_split(dataset, test_size=0.25, random_state=1)
        test_dataset, valid_dataset = train_test_split(test_dataset, test_size=0.2, random_state=1)

        # Normalization
        self.scaler = MinMaxScaler()
        self.scaler.fit(train_dataset[['target']])
        train_dataset['target'] = self.scaler.transform(train_dataset[['target']])
        test_dataset['target'] = self.scaler.transform(test_dataset[['target']])
        valid_dataset['target'] = self.scaler.transform(valid_dataset[['target']])

        # Create datasets
        training_set = CustomDataset(train_dataset, self.tokenizer, self.max_length)
        test_set = CustomDataset(test_dataset, self.tokenizer, self.max_length)
        valid_set = CustomDataset(valid_dataset, self.tokenizer, self.max_length)

        # Create data loaders
        train_params = {
            'batch_size': self.train_batch_size,
            'shuffle': True
        }
        test_params = {
            'batch_size': self.test_batch_size,
            'shuffle': False
        }
        self.training_loader = DataLoader(training_set, **train_params)
        self.test_loader = DataLoader(test_set, **test_params)
        self.valid_loader = DataLoader(valid_set, **test_params)

    def train(self, model, nb_epochs):
        optimizer = torch.optim.Adam(params=model.parameters(), lr=self.learning_rate)

        for epoch in range(nb_epochs):
            # Training part
            model.train()
            training_abs_err, train_loss = 0, 0
            for _, data in tqdm(
                        enumerate(self.training_loader),
                        total=len(self.training_loader),
                        desc=f"Epoch {epoch + 1}",
                        miniters=10
                    ):
                optimizer.zero_grad()

                targets = data['targets'].to(device, dtype=torch.float)

                outputs = model.eval_with_dict(data)

                # Compute loss and other metrics
                loss = self.loss(outputs, targets)
                train_loss += loss.item()
                training_abs_err += compute_abs_error(outputs, targets, self.scaler)

                # compute gradients and perform one optimization step
                loss.backward()
                optimizer.step()

            # End of epoch: run validation & print train and validation scores
            # Validation part
            model.eval()
            valid_abs_error, valid_loss = 0, 0
            for data in self.valid_loader:
                targets = data['targets'].to(device, dtype=torch.float)
                outputs = model.eval_with_dict(data)
                loss = self.loss(outputs, targets)
                valid_loss += loss.item()
                valid_abs_error += compute_abs_error(outputs, targets, self.scaler)

            # Scores
            valid_mae = valid_abs_error / len(self.valid_loader.dataset)
            valid_loss = valid_loss / len(self.valid_loader.dataset)

            train_mae = training_abs_err / len(self.training_loader.dataset)
            train_loss = train_loss / len(self.training_loader.dataset)
            print((
                f"Epoch {epoch + 1} / {nb_epochs}, Loss: {train_loss:.3f}, MAE: {train_mae:.1f} "
                f"- Validation Loss: {valid_loss:.3f}, Validation MAE: {valid_mae:.1f}"))

    

        # Return best (checkpointed) model
        model.load_state_dict(torch.load('checkpoint.pt'))
        return model


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)[y_true > 0] / (y_true + 1e-7)[y_true > 0])) * 100


def max_absolute_error(y_true, y_pred):
    return np.max(np.abs((y_true - y_pred)))


def compute_metrics(y_test, y_pred):
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    # print(f"r2: {r2_score(y_test, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
    print(f"MedAE: {median_absolute_error(y_test, y_pred)}")
    print(f"MaxAE: {max_absolute_error(y_test, y_pred)}")
    print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred)}")

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
model = BERTForRegression(
        layer_size=5,
        dropout=0.5
    )
 
model.to(device)

BERTForRegression(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [48]:
 # Load data
    df = pd.read_excel("data_sizing_bin1.xlsx")
    df=df[df['ProductName']=='Web Check-in']
    df = df[["RequirementDescription", "RecommendedPricing", "Title",'PCPCategory',"Sizing"]]
    #df["Sizing_bins"].fillna("Very Large", inplace=True)  # To be checked
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.columns = ['text', 'RecommendedPricing', 'Title','PCPCategory','target']

    # Target pre-processing
    df.target = np.log(df.target + 1)

    # Text preprocessing
    df['text'] = df['text'].str.lower()
    # Remove some useless content
    # df['text'] = df['text'].str.replace('winaproach', '', regex=False)
    # df['text'] = df['text'].str.replace('this task has been automatically created', '', regex=False)
    # Remove URLs
    df['text'] = df['text'].str.replace(r'http\S+', '', regex=True)
    # Remove HTML tags
    df['text'] = df['text'].str.replace(r'<.*?>', ' ', regex=True)
    # Remove punctuation - is it useful?
    df['text'] = df['text'].str.replace('-', ' ', regex=False)
    df['text'] = df['text'].str.translate(str.maketrans('', '', string.punctuation))

In [49]:
    # Training
    trainer = Trainer(
        dataset=df,
        tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'),
        learning_rate=0.01,
        patience=5,
        train_batch_size=5,
        test_batch_size=1
    )

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
trainer.train(model, 150)

Epoch 2:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1 / 150, Loss: 0.031, MAE: 78.1 - Validation Loss: 0.068, Validation MAE: 10.5


Epoch 3:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 2 / 150, Loss: 0.031, MAE: 78.2 - Validation Loss: 0.068, Validation MAE: 10.5


Epoch 4:   0%|          | 0/4 [00:00<?, ?it/s]

...

Epoch 23 / 150, Loss: 0.033, MAE: 77.7 - Validation Loss: 0.068, Validation MAE: 10.5


Epoch 25:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 24 / 150, Loss: 0.031, MAE: 77.6 - Validation Loss: 0.068, Validation MAE: 10.5


Epoch 26:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 25 / 150, Loss: 0.026, MAE: 76.5 - Validation Loss: 0.068, Validation MAE: 10.5


In [ ]:
    # Evaluation
    preds, targets = model.eval_with_loader(trainer.test_loader)
    preds_untr = np.exp(
        trainer.scaler.inverse_transform([preds])[0,:]
    ) - 1
    targets_untr = np.exp(
        trainer.scaler.inverse_transform([targets])[0,:]
    ) - 1
    compute_metrics(preds_untr, targets_untr)